In [500]:
#ライブラリをインポート
import os #OSに依存する様々な機能を利用するためのモジュール(ファイルやディレクトリ操作など)
import re #正規表現を利用するためのモジュール
import csv  #csvファイルを扱うためのモジュール
import math #数学的計算のためのモジュール
import matplotlib.pyplot as plt #グラフ描画のためのモジュール
import numpy as np  #多次元配列計算のためのモジュール
import pandas as pd #データフレームを扱うためのモジュール
from scipy.stats import skew, kurtosis  #歪度と尖度を調べるためのモジュール
from sklearn.model_selection import train_test_split  #データをトレーニング用とテスト用に分けるためのモジュール
from sklearn import preprocessing #データを正規化するためのモジュール
from sklearn.preprocessing import StandardScaler  #データを標準化するためのモジュール
from sklearn.linear_model import LinearRegression #線型回帰
from sklearn.svm import SVC #サポートベクターマシン
from sklearn.ensemble import RandomForestClassifier #ランダムフォレスト
from sklearn.neighbors import KNeighborsClassifier  #k-近傍法
from sklearn.metrics import accuracy_score  #機械学習モデルの性能評価のためのモジュール

In [501]:
#定数を定義
BINS = 4000  #ヒストグラムのビンの数
EPSILON = .00001  #スムージングパラメータ
UPPER_LIMIT = 1.1 #静止区間の上限
LOWER_LIMIT = 0.9 #静止区間の加減
STATIONARY_INTERVALS = 5  #静止区間除去のサンプルの間隔(静止区間が何サンプル連続したら除去するか)
TRAIN_SIZE = 0.8  #ランダムフォレストのトレーニングデータの割合
N_ESTIMATORS = 100

In [502]:
#ディレクトリ内のデータセットのファイル名と周波数を取得する関数
def get_Hz_and_filename(path: str) -> list[int, str]:
    filename = os.listdir(path) #引数のパスのディレクトリの中のファイル名一覧を取得
    Hz_and_filename=[]  #ファイル名と周波数を格納するリストを宣言

    for file in filename:
        Hz = re.search(r'\d+', file)    #正規表現を用いてファイル名の中で一番最初に出てくる数字(周波数)を取得
        if Hz:  #数字の入っていないファイル名があるとエラーを吐くので、このif文でチェックする
            Hz_and_filename.append([int(Hz.group(0)), file])    #ファイル名と周波数を格納

    return Hz_and_filename

In [503]:
#ファイル名と周波数を分けて出力する関数
def divide_Hz_and_filename(Hz_and_filename: list[int, str]) -> tuple[list[int], list[str]]:
    Hz = []
    filename = []
    for row in Hz_and_filename:
      Hz.append(row[0])
      filename.append(row[1])

    return Hz, filename

In [504]:
#加速度データのCSVファイルから3軸加速度を取得する関数
def get_acceleration(filename: str) -> tuple[list[float], list[float], list[float]]:
    AccX, AccY, AccZ = [], [], []
    with open(filename) as f:
        reader = csv.reader(f)
        for row in reader:
            AccX.append(float(row[2]))
            AccY.append(float(row[3]))
            AccZ.append(float(row[4]))

    return AccX, AccY, AccZ

In [505]:
#静止区間を除去する関数
def remove_stationary_intervals(AccX: list[float], AccY: list[float], AccZ: list[float]) -> list[float]:
    #各軸の加速度の平均を求める
    AvgAccX = sum(AccX) / len(AccX)
    AvgAccY = sum(AccY) / len(AccY)
    AvgAccZ = sum(AccZ) / len(AccZ)

    AvgResultantAcc = math.sqrt(AvgAccX ** 2 + AvgAccY ** 2 + AvgAccZ ** 2) #重力加速度の推定値=合成加速度の平均を求める

    ResultantAcc = [math.sqrt(x ** 2 + y ** 2 + z ** 2) for x, y, z in zip(AccX, AccY, AccZ)]   #各時刻の合成加速度を求める

    #各時刻の合成加速度から静止区間(重力加速度の推定値に近い値が一定以上以上連続している区間)を除去する
    i = 0 #ループ変数
    counter = 0 #静止区間がSTATIONARY_INTERVALS分続いているかをカウントする変数
    while i < len(ResultantAcc):
        if AvgResultantAcc * LOWER_LIMIT < ResultantAcc[i] < AvgResultantAcc * UPPER_LIMIT:   #平均のLOWER_LIMIT倍~UPPER_LIMIT倍の範囲を調べる
            counter += 1    #範囲内ならカウントを増やす
            if counter == STATIONARY_INTERVALS: #カウントがSTATIONARY_INTERVALSに達したらその区間を削除
                del ResultantAcc[i+1-STATIONARY_INTERVALS:i+1]    #スライスでは選択範囲の開始位置startと終了位置stopを[start:stop]のように書くとstart <= x < stopの範囲が選択される #start番目の値は含まれるがstop番目の値は含まれない
                counter = 0 #カウンターをリセット
                i -= STATIONARY_INTERVALS   #削除した分インデックスがズレるので補正する
        else:
            counter = 0 #カウンターをリセット
        i += 1

    return ResultantAcc  #静止区間を除去した後のリストを返す

In [506]:
#連続する2サンプルの差分を取る関数
def calculate_differences_of_acceleration(ResultantAcc: list[float]) -> list[float]:
    DifferenceAcc = [math.fabs(ResultantAcc[i + 1] * 100000 - ResultantAcc[i] * 100000) for i in range(len(ResultantAcc) - 1)]  #100000倍して誤差を取る
    return DifferenceAcc

In [507]:
#KLダイバージェンス関数 #引数として与える2つの分布は非負の値の集合でなければならないことに注意
def KL_divergence(a: list[float], b: list[float]) -> float:
    min_value = min(min(a), min(b)) #a,bの最小値の小さい方
    max_value = max(max(a), max(b)) #a,bの最大値の大きい方

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()

    #KLダイバージェンスの値を返す
    return np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, b_hist)])

In [508]:
#JSダイバージェンス関数 #引数として与える2つの分布は非負の値の集合でなければならないことに注意
def JS_divergence(a: list[float], b: list[float]) -> float:
    min_value = min(min(a), min(b)) #a,bの最小値の小さい方
    max_value = max(max(a), max(b)) #a,bの最大値の大きい方

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()

    #2つの分布の平均値を求める
    mean_hist = (a_hist + b_hist) / 2.0

    #平均とそれぞれの分布のKLダイバージェンスを算出
    kl_a = np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, mean_hist)])
    kl_b = np.sum([ai * np.log(ai / bi) for ai, bi in zip(b_hist, mean_hist)])

    #JSダイバージェンスの値を返す
    return (kl_a + kl_b) / 2.0

In [509]:
#データフレームの各行の中で2番目に小さい値が格納されている場所を調べる関数(最小値は同じ確率分布同士の0.0)
def get_index_and_columns_of_second_smallest(df: pd.DataFrame) -> list[str, str]:
    index_and_columns_of_second_smallest = []  #データフレームの中で2番目に小さい値が格納されている場所のインデックス名とカラム名を格納する変数
    for i in range(len(df)):
        sorted_row = df.iloc[i].sort_values()   #.ilocでデータフレームの要素を行、列の番号の添字で指定する    #各行の要素を昇順に並び替える
        second_smallest_columns = sorted_row.index[1] #各行の2番目に小さい値が格納されているカラム[1]の名前を取得
        #second_smallest_label = df.columns.get_loc(second_smallest_index)
        index_and_columns_of_second_smallest.append((df.index[i], second_smallest_columns))    #インデックスとカラムのラベル名の組を二次元配列に追加
    return index_and_columns_of_second_smallest

In [510]:
#推定精度を算出する関数
def calculate_accuracy(index_and_columns_of_second_smallest: list[str, str]) -> tuple[float, list[int]]:
    counter = 0
    error_index_list = []
    for i in range(len(index_and_columns_of_second_smallest)):
        #インデックスとカラムのラベル名が同じならばカウンターを1増やす
        if index_and_columns_of_second_smallest[i][0] == index_and_columns_of_second_smallest[i][1]:
            counter += 1
        else:
            error_index_list.append(i)
            print(f"間違ってるやつは{i}番目の{index_and_columns_of_second_smallest[i][0]}と{index_and_columns_of_second_smallest[i][1]}です")

    return (counter / len(index_and_columns_of_second_smallest)) * 100, error_index_list  #精度を100分率で返す

In [511]:
#入力された加速度の差分のリストからヒストグラムを作る関数（ビンの範囲が全加速度の差分データの最小値〜最大値）
def create_histogram(DifferenceAcc_list: list[float]) -> np.histogram:
    min_value = min(map(lambda x:max(x), DifferenceAcc_list))   #入力されたリストの中で最も小さい数
    max_value = max(map(lambda x:max(x), DifferenceAcc_list))   #入力されたリストの中で最も大きい数

    DifferenceAcc_hist = np.zeros((len(DifferenceAcc_list), BINS), dtype=float)

    for i in range(len(DifferenceAcc_list)):
        DifferenceAcc_hist[i], _ = np.histogram(DifferenceAcc_list[i], bins=BINS, range=(min_value, max_value)) #ヒストグラムを作成し、同じ数のビンで区切る
    return DifferenceAcc_hist

In [512]:
#入力された加速度の差分のリストからヒストグラムを作る関数（ビンの範囲が各加速度の差分データの最小値〜最大値）
def create_histogram2(DifferenceAcc_list: list[float]) -> np.histogram:
    DifferenceAcc_hist = np.zeros((len(DifferenceAcc_list), BINS), dtype=float)
    for i in range(len(DifferenceAcc_list)):
        min_value = min(DifferenceAcc_list[i])
        max_value = max(DifferenceAcc_list[i])
        DifferenceAcc_hist[i], _ = np.histogram(DifferenceAcc_list[i], bins=BINS, range=(min_value, max_value)) #ヒストグラムを作成し、同じ数のビンで区切る
    return DifferenceAcc_hist

In [513]:
#KLダイバージェンスとJSダイバージェンス算出の一連の流れを自動化した関数
def KL_and_JS(path: str):
    Hz_and_filename = get_Hz_and_filename(path)
    Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_Hz_and_filename(Hz_and_filename)
    Hz = [str(hz) + "Hz" for hz in Hz]  #周波数の値+"Hz"のリストを作りデータフレームのラベルに用いる

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []
    resultKLD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化
    resultJSD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化
    error_index_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc))

    #KLダイバージェンスの値を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            resultKLD[i][j] = KL_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

    #JSダイバージェンスの値を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            resultJSD[i][j] = JS_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

    #結果を出力
    df_KLD = pd.DataFrame(resultKLD, index=Hz, columns=Hz)
    display(df_KLD)
    accuracyKLD, error_index_list = calculate_accuracy(get_index_and_columns_of_second_smallest(df_KLD))
    for i in range(len(error_index_list)):
        print(filename[error_index_list[i]])
    print(f"KLダイバージェンスによる推定精度は{accuracyKLD}%です")

    df_JSD = pd.DataFrame(resultJSD, index=Hz, columns=Hz)
    display(df_JSD)
    accuracyJSD, error_index_list = calculate_accuracy(get_index_and_columns_of_second_smallest(df_JSD))
    for i in range(len(error_index_list)):
        print(filename[error_index_list[i]])
    print(f"JSダイバージェンスによる推定精度は{accuracyJSD}%です")

In [426]:
#ランダムフォレストによる機械学習モデル構築と性能評価までを自動化した関数
def random_forest(path: str):
    Hz_and_filename = get_Hz_and_filename(path)
    Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_Hz_and_filename(Hz_and_filename)

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc))

    DifferenceAcc_hist = create_histogram2(DifferenceAcc_list)

    x_train, x_test, y_train, y_test = train_test_split(DifferenceAcc_hist, Hz, train_size = TRAIN_SIZE, shuffle = True)

    # 学習する
    clf = RandomForestClassifier(n_estimators=N_ESTIMATORS, random_state=1234)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print("正解率 = ", accuracy_score(y_test, y_pred))

In [427]:
#ランダムフォレストによる機械学習モデル構築と性能評価までを自動化した関数
def k_neighbors(path: str):
    Hz_and_filename = get_Hz_and_filename(path)
    Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_Hz_and_filename(Hz_and_filename)

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc))

    DifferenceAcc_hist = create_histogram2(DifferenceAcc_list)
    scaler = StandardScaler()
    scaler.fit(DifferenceAcc_hist)
    newdata = scaler.transform(DifferenceAcc_hist)

    x_train, x_test, y_train, y_test = train_test_split(newdata, Hz, train_size = TRAIN_SIZE, shuffle = True)

    # 学習する
    clf = KNeighborsClassifier()
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print("正解率 = ", accuracy_score(y_test, y_pred))

In [514]:
path = "my_walk_data/"

In [493]:
path3 = "experiment_data/"

In [494]:
KL_and_JS(path3)

,100Hz,100Hz,90Hz,90Hz,80Hz,80Hz,70Hz,70Hz,60Hz,60Hz,50Hz,50Hz,40Hz,40Hz,30Hz,30Hz,20Hz,20Hz,10Hz,10Hz
100Hz,0.000000,0.043061,0.034565,0.066580,0.060987,0.093568,0.131581,0.176389,0.232583,0.286724,0.333595,0.383428,0.479857,0.547302,0.678939,0.766316,1.007711,1.085572,1.996695,1.946237
100Hz,0.053110,0.000000,0.026806,0.032534,0.055605,0.059555,0.089563,0.111396,0.164992,0.195496,0.254947,0.294948,0.370174,0.442193,0.577519,0.649598,0.893752,0.968922,1.859875,1.821945
90Hz,0.042236,0.031773,0.000000,0.036312,0.037407,0.048518,0.076883,0.113032,0.163286,0.205909,0.248141,0.296745,0.387952,0.448091,0.581712,0.661896,0.905421,0.981779,1.810465,1.829994
90Hz,0.101761,0.046331,0.049851,0.000000,0.048862,0.028215,0.045425,0.063586,0.096839,0.130789,0.170740,0.205167,0.289416,0.343969,0.472459,0.549339,0.791768,0.864926,1.662202,1.692266
80Hz,0.067260,0.049041,0.030298,0.029784,0.000000,0.053136,0.078951,0.094438,0.146498,0.173644,0.231389,0.270873,0.348406,0.418703,0.555848,0.625567,0.870981,0.946013,1.832198,1.794626
80Hz,0.139408,0.078622,0.065868,0.027937,0.087579,0.000000,0.071683,0.059657,0.116834,0.118457,0.172520,0.206065,0.264713,0.335159,0.460835,0.522494,0.766038,0.839361,1.705491,1.676809
70Hz,0.179123,0.103997,0.089525,0.038026,0.101987,0.061547,0.000000,0.053722,0.096061,0.099487,0.152033,0.182880,0.242214,0.309523,0.439190,0.495934,0.741536,0.812892,1.674376,1.651846
70Hz,0.293620,0.167135,0.169244,0.077155,0.160106,0.076277,0.079328,0.000000,0.070323,0.068262,0.094360,0.116900,0.168702,0.223120,0.344875,0.401784,0.640547,0.709360,1.554387,1.529905
60Hz,0.363409,0.221200,0.214884,0.101909,0.228167,0.119904,0.119366,0.049600,0.000000,0.052219,0.088274,0.111049,0.138848,0.203106,0.326754,0.374568,0.617793,0.686826,1.521122,1.505398
60Hz,0.493415,0.304641,0.322735,0.163365,0.313198,0.153204,0.152189,0.069081,0.075289,0.000000,0.055292,0.058711,0.092699,0.137294,0.245346,0.298309,0.531380,0.597673,1.415585,1.401701


間違ってるやつは0番目の100Hzと90Hzです
間違ってるやつは1番目の100Hzと90Hzです
間違ってるやつは2番目の90Hzと100Hzです
間違ってるやつは3番目の90Hzと80Hzです
間違ってるやつは4番目の80Hzと90Hzです
間違ってるやつは5番目の80Hzと90Hzです
間違ってるやつは6番目の70Hzと90Hzです
間違ってるやつは7番目の70Hzと60Hzです
間違ってるやつは8番目の60Hzと70Hzです
間違ってるやつは9番目の60Hzと50Hzです
間違ってるやつは10番目の50Hzと60Hzです
間違ってるやつは11番目の50Hzと40Hzです
間違ってるやつは15番目の30Hzと20Hzです
mem-100Hz-20230117-1716.csv
mem-100Hz-20230117-1655.csv
mem-90Hz-20230117-1716.csv
mem-90Hz-20230117-1655.csv
mem-80Hz-20230117-1716.csv
mem-80Hz-20230117-1655.csv
mem-70Hz-20230117-1716.csv
mem-70Hz-20230117-1655.csv
mem-60Hz-20230117-1716.csv
mem-60Hz-20230117-1655.csv
mem-50Hz-20230117-1716.csv
mem-50Hz-20230117-1655.csv
mem-30Hz-20230117-1655.csv
KLダイバージェンスによる推定精度は35.0%です


,100Hz,100Hz,90Hz,90Hz,80Hz,80Hz,70Hz,70Hz,60Hz,60Hz,50Hz,50Hz,40Hz,40Hz,30Hz,30Hz,20Hz,20Hz,10Hz,10Hz
100Hz,0.000000,0.007789,0.006240,0.016596,0.011256,0.022548,0.030200,0.045962,0.059790,0.076225,0.089829,0.106644,0.133851,0.151853,0.187573,0.208759,0.267005,0.284389,0.429939,0.435128
100Hz,0.007789,0.000000,0.003530,0.005944,0.005782,0.010237,0.015173,0.025448,0.036547,0.049422,0.062482,0.078571,0.102035,0.121355,0.158897,0.177603,0.239772,0.257263,0.408388,0.414889
90Hz,0.006240,0.003530,0.000000,0.007083,0.004387,0.010037,0.014964,0.027324,0.038129,0.052777,0.064499,0.080743,0.107079,0.124180,0.161337,0.181738,0.243115,0.260944,0.410578,0.417065
90Hz,0.016596,0.005944,0.007083,0.000000,0.005222,0.003599,0.005907,0.012328,0.019339,0.030378,0.039736,0.053629,0.077050,0.093259,0.129977,0.150140,0.214493,0.232229,0.386056,0.393509
80Hz,0.011256,0.005782,0.004387,0.005222,0.000000,0.008811,0.012259,0.021672,0.031866,0.044537,0.057525,0.073856,0.097340,0.117210,0.156064,0.174351,0.237509,0.255267,0.407643,0.414453
80Hz,0.022548,0.010237,0.010037,0.003599,0.008811,0.000000,0.006793,0.010206,0.017976,0.025952,0.036872,0.051349,0.070933,0.089951,0.127447,0.144845,0.210003,0.227965,0.385044,0.392579
70Hz,0.030200,0.015173,0.014964,0.005907,0.012259,0.006793,0.000000,0.008331,0.014175,0.021607,0.031546,0.045836,0.064855,0.084221,0.122794,0.139504,0.206128,0.223995,0.382573,0.390745
70Hz,0.045962,0.025448,0.027324,0.012328,0.021672,0.010206,0.008331,0.000000,0.006661,0.010226,0.015530,0.026385,0.041859,0.058185,0.094070,0.111484,0.178467,0.196001,0.358515,0.367137
60Hz,0.059790,0.036547,0.038129,0.019339,0.031866,0.017976,0.014175,0.006661,0.000000,0.008154,0.014233,0.025065,0.036114,0.054501,0.091319,0.106907,0.176075,0.193862,0.357902,0.366997
60Hz,0.076225,0.049422,0.052777,0.030378,0.044537,0.025952,0.021607,0.010226,0.008154,0.000000,0.006406,0.011120,0.020463,0.034395,0.066217,0.083318,0.151068,0.168683,0.336046,0.345419


間違ってるやつは0番目の100Hzと90Hzです
間違ってるやつは1番目の100Hzと90Hzです
間違ってるやつは2番目の90Hzと100Hzです
間違ってるやつは3番目の90Hzと80Hzです
間違ってるやつは4番目の80Hzと90Hzです
間違ってるやつは5番目の80Hzと90Hzです
間違ってるやつは6番目の70Hzと90Hzです
間違ってるやつは7番目の70Hzと60Hzです
間違ってるやつは8番目の60Hzと70Hzです
間違ってるやつは9番目の60Hzと50Hzです
間違ってるやつは10番目の50Hzと60Hzです
間違ってるやつは11番目の50Hzと40Hzです
間違ってるやつは12番目の40Hzと50Hzです
mem-100Hz-20230117-1716.csv
mem-100Hz-20230117-1655.csv
mem-90Hz-20230117-1716.csv
mem-90Hz-20230117-1655.csv
mem-80Hz-20230117-1716.csv
mem-80Hz-20230117-1655.csv
mem-70Hz-20230117-1716.csv
mem-70Hz-20230117-1655.csv
mem-60Hz-20230117-1716.csv
mem-60Hz-20230117-1655.csv
mem-50Hz-20230117-1716.csv
mem-50Hz-20230117-1655.csv
mem-40Hz-20230117-1716.csv
JSダイバージェンスによる推定精度は35.0%です


In [433]:
k_neighbors(path)

正解率 =  0.23529411764705882


In [476]:
path2 = "my_walk_data(only_3classes)/"

In [498]:
random_forest(path3)

正解率 =  0.25


In [516]:
#個々実験用
Hz_and_filename = get_Hz_and_filename(path)
Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
Hz, filename = divide_Hz_and_filename(Hz_and_filename)

#使う変数を宣言
AccX, AccY, AccZ = [], [], []
ResultantAcc = []
DifferenceAcc_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
for i in filename:
    AccX, AccY, AccZ = get_acceleration(path+i)
    ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
    DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc))

DifferenceAcc_hist = create_histogram2(DifferenceAcc_list)

In [517]:
for i in range(len(DifferenceAcc_list)):
  print(f"{Hz[i]}と{len(DifferenceAcc_list[i])}と{filename[i]}\n")

100と111367とwalk100Hz-20230318-200648365.csv

100と121304とwalk100Hz-20230315-195146474.csv

100と192093とwalk100Hz-20230314-161440229.csv

100と94482とwalk100Hz-20230312-164304804.csv

100と139361とwalk100Hz-20230312-123621260.csv

100と83388とwalk100Hz-20230312-120722989.csv

100と153567とwalk100Hz-20230310-173457927.csv

100と123136とwalk100Hz-20230309-180537652.csv

100と70227とwalk100Hz-20230309-171155001.csv

100と78363とwalk100Hz-20230303-111623870.csv

100と50417とwalk100Hz-20230302-165446613.csv

100と225375とwalk100Hz-20230302-115943109.csv

100と74400とwalk100Hz-20230301-203316681.csv

100と74082とwalk100Hz-20230228-195310844.csv

100と85304とwalk100Hz-20230227-202328538.csv

100と177817とwalk100Hz-20230227-194442583.csv

90と85058とwalk90Hz-20191007-204039412.csv

90と129895とwalk90Hz-20191006-075256268.csv

90と136157とwalk90Hz-20191005-220602624.csv

90と99138とwalk90Hz-20191004-065122936.csv

90と91207とwalk90Hz-20191004-043737805.csv

80と81550とwalk80Hz-20191007-204459666.csv

80と133702とwalk80Hz-20191006-075710

In [519]:
kinbou = []

for i in range(100):
    scaler = StandardScaler()
    scaler.fit(DifferenceAcc_hist)
    newdata = scaler.transform(DifferenceAcc_hist)

    x_train, x_test, y_train, y_test = train_test_split(newdata, Hz, train_size = TRAIN_SIZE, shuffle = True)

    # 学習する
    clf = KNeighborsClassifier()
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    kinbou.append(accuracy_score(y_test, y_pred))

In [520]:
mink = sum(kinbou) / len(kinbou)
print(mink)

0.36117647058823493


In [521]:
#尖度
DifferenceAcc_kurtosis = np.zeros(len(DifferenceAcc_list))
for i in range(len(DifferenceAcc_hist)):
    DifferenceAcc_kurtosis[i] = kurtosis(DifferenceAcc_hist[i])

In [522]:
#歪度
DifferenceAcc_skewness = np.zeros(len(DifferenceAcc_list))
for i in range(len(DifferenceAcc_hist)):
    DifferenceAcc_skewness[i] = skew(DifferenceAcc_hist[i])

In [523]:
#分散
histogram_var = np.zeros(len(DifferenceAcc_list))
for i in range(len(DifferenceAcc_hist)):
    histogram_var[i] = np.var(DifferenceAcc_list[i])
    #histogram_var[i] = np.var(DifferenceAcc_list[i])/len(DifferenceAcc_list[i])


In [524]:
DifferenceAcc_kurtosis

array([ 65.79260793,  64.82257625, 258.78413727,  93.46574104,
       154.24801941,  80.93957313, 288.24697085, 377.00916196,
        72.59707352,  62.95580836,  81.25694827, 349.01347817,
        51.57744453,  64.84587053, 200.34497229, 443.17660575,
        87.12374853,  69.43038122, 146.09371484,   7.51203189,
       111.43181252, 102.32107071, 119.7497685 , 229.58066698,
        10.06647372,  41.26030917, 123.17720033,  94.97312889,
       202.28666514,   9.83164446,  19.02180259,  91.13200913,
        54.74801277, 148.22079333,  24.98781809,  21.51257551,
        50.44863182,  65.38321323,  57.00326428, 174.31118732,
        72.18960051, 182.83226125, 302.32873474,  54.76574919,
        10.89314727,  74.60272903, 248.75797663,  41.86488221,
        41.57931913, 108.15790465, 288.57099289,  32.31143013,
        70.65151567,  97.25924627,  26.21060744,  30.09548905,
        81.10856659,  24.17250631, 128.7460289 ,  17.35455014,
        12.19108975,  51.74359204,  18.79212836, 114.14

In [525]:
DifferenceAcc_skewness

array([ 7.87615704,  7.52723461, 14.95354541,  9.26587951, 11.64261595,
        8.62596173, 15.89487234, 18.45369053,  8.17061332,  7.72880134,
        8.59598098, 17.71225195,  7.08869387,  7.64948758, 13.03714102,
       19.7380418 ,  8.64306515,  7.93061426, 11.26660011,  2.93191233,
        9.99544253,  9.17405582, 10.08316242, 14.15095597,  3.29574055,
        6.17388856, 10.09656553,  8.92879779, 13.25407645,  3.25026161,
        4.23764072,  8.68937049,  7.10496697, 11.27282309,  4.77748362,
        4.60263347,  6.63493834,  7.75831598,  7.28745365, 12.27418839,
        8.05399356, 12.53584859, 16.39078498,  7.16745364,  3.37080515,
        8.29696719, 14.81380602,  6.41906919,  6.23256143,  9.44014958,
       15.61564787,  5.42920026,  7.65778095,  9.14357827,  4.97723171,
        5.11854252,  7.95711152,  4.80441082, 10.49666726,  4.09466108,
        3.52536242,  6.38576261,  4.27335137,  9.7846631 ,  2.0979605 ,
        2.74510072,  1.74037599,  2.99015031,  8.34864977,  4.13

In [526]:
histogram_var

array([7.93053204e+15, 4.84575364e+16, 9.45944237e+16, 2.89494524e+17,
       7.46482330e+16, 4.33553387e+17, 1.79046686e+16, 5.59904623e+15,
       7.29483273e+15, 9.30644499e+15, 1.25765653e+16, 9.00768677e+15,
       3.94728075e+15, 2.67822934e+16, 2.23368926e+16, 9.51268817e+15,
       3.81116630e+16, 2.90924690e+16, 1.39545526e+16, 3.48645305e+15,
       2.92023766e+16, 3.67312680e+16, 3.05854059e+16, 1.53699257e+16,
       4.32206679e+15, 1.65423860e+16, 5.01081293e+16, 4.75307546e+16,
       1.73272513e+16, 6.40573222e+15, 1.55688716e+16, 9.16315759e+16,
       7.77139200e+16, 2.27248842e+16, 2.06558073e+16, 1.97647454e+16,
       9.84446521e+16, 2.66817174e+17, 6.06003748e+17, 3.04520075e+17,
       6.46097431e+17, 3.45415214e+16, 1.27885002e+16, 2.03523389e+16,
       3.17393256e+16, 3.21383434e+16, 1.50658412e+16, 9.73358158e+15,
       2.64536153e+16, 3.23730918e+16, 1.42029591e+16, 1.42681531e+17,
       9.29818881e+16, 3.41177076e+16, 1.75087046e+16, 4.18296865e+16,
      

In [527]:
X_new = np.concatenate((DifferenceAcc_kurtosis.reshape(-1, 1), DifferenceAcc_skewness.reshape(-1, 1), histogram_var.reshape(-1, 1)), axis=1)
X = np.concatenate((DifferenceAcc_hist, X_new), axis=1)

In [528]:
X_new[0]

array([6.57926079e+01, 7.87615704e+00, 7.93053204e+15])

In [529]:
len(X)

84

In [530]:
#ヒストグラム+尖度+歪度+分散
x_train, x_test, y_train, y_test = train_test_split(X, Hz, train_size = TRAIN_SIZE, shuffle = True)
clf = RandomForestClassifier(random_state=1234)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("正解率 = ", accuracy_score(y_test, y_pred))

正解率 =  0.4117647058823529


In [531]:
#ヒストグラム
x_train, x_test, y_train, y_test = train_test_split(DifferenceAcc_hist, Hz, train_size = TRAIN_SIZE, shuffle = True)
clf = RandomForestClassifier(random_state=1234)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("正解率 = ", accuracy_score(y_test, y_pred))

正解率 =  0.35294117647058826


In [532]:
#尖度+歪度+分散
x_train, x_test, y_train, y_test = train_test_split(X_new, Hz, train_size = TRAIN_SIZE, shuffle = True)
clf = RandomForestClassifier(random_state=1234)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("正解率 = ", accuracy_score(y_test, y_pred))

正解率 =  0.35294117647058826


In [533]:
hist_kurts = []
only_hist = []
kurtskewvar = []

for i in range(100):
    #ヒストグラム+尖度+歪度+分散
    x_train, x_test, y_train, y_test = train_test_split(X, Hz, train_size = TRAIN_SIZE, shuffle = True)
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    hist_kurts.append(accuracy_score(y_test, y_pred))

    #ヒストグラム
    x_train, x_test, y_train, y_test = train_test_split(DifferenceAcc_hist, Hz, train_size = TRAIN_SIZE, shuffle = True)
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    only_hist.append(accuracy_score(y_test, y_pred))

    #尖度+歪度+分散
    x_train, x_test, y_train, y_test = train_test_split(X_new, Hz, train_size = TRAIN_SIZE, shuffle = True)
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    kurtskewvar.append(accuracy_score(y_test, y_pred))

In [535]:
min_hist_kurts = sum(hist_kurts) / len(hist_kurts)
min_only_hist = sum(only_hist) / len(only_hist)
min_kurtskewvar = sum(kurtskewvar) / len(kurtskewvar)
print(f"ヒスト+尖度+歪度+分散の精度：{min_hist_kurts}\nヒストグラムのみの精度:{min_only_hist}\n尖度+歪度+分散だけの精度:{min_kurtskewvar}")

ヒスト+尖度+歪度+分散の精度：0.4923529411764704
ヒストグラムのみの精度:0.47117647058823503
尖度+歪度+分散だけの精度:0.21411764705882316
